In [1]:
import bittensor
import torch 
from transformers import AutoTokenizer
import numpy as np

In [2]:
logging=bittensor.logging(debug=True)
dataload = bittensor.dataloader(max_corpus_size=10000)

2021-09-17 12:21:36.520 |     SUCCESS      | Set debug:          ON
2021-09-17 12:21:36.521 |     SUCCESS      | Set trace:          OFF
2021-09-17 12:21:36.521 |     SUCCESS      | Set record log:     OFF


In [3]:
data = dataload.dataloader(epoch_length=100)

2021-09-17 12:21:38.745 |     SUCCESS      | Retrieving a dataset files from the IPFS gateway...
2021-09-17 12:21:39.182 |     SUCCESS      | Loaded folder:      QmXwfPoh2QFYqC6cYcW8kzyd9ruFfhnUi2kVBkdhawjUzj
2021-09-17 12:21:41.307 |     SUCCESS      | Loaded folder:      QmRjFNn3XpYMycVzTE4YcVcxk45vNZcTAjKmtEqPLKwAWd
2021-09-17 12:21:41.329 |     SUCCESS      | Loaded:             Wikitext_TōshirōHitsugaya.txt
2021-09-17 12:21:41.861 |     SUCCESS      | Downloaded:         Wikitext_ElmerStricklett.txt
2021-09-17 12:21:41.862 |     SUCCESS      | Saved:              Wikitext_ElmerStricklett.txt


In [4]:
from transformers import AutoModel, GPT2Model

model = GPT2Model.from_pretrained("gpt2")


In [5]:
data_iter = iter(data)
test = next(data_iter)

In [9]:
import torch.nn.functional as F

class server(torch.nn.Module):
    def __init__(self, pretrained,pre_dimension,final_dim ):
        super(server, self).__init__()
        self.pretrained = pretrained
        self.final_dim =  final_dim
        self.pre_dimension = pre_dimension
        #self.mapping = torch.nn.Linear( pre_dimension, final_dim)
        self.decoder = torch.nn.Linear( final_dim, bittensor.__vocab_size__ , bias=False)
        self.loss_fct = torch.nn.CrossEntropyLoss()
        
    def forward(self, inputs):
        pre_hidden = self.pretrained(inputs).last_hidden_state
        padding_l = (self.final_dim-self.pre_dimension)//2
        padding_r = (self.final_dim-self.pre_dimension) - padding_l
        encoded_hidden = F.pad(pre_hidden, (padding_l, padding_r),  "constant", 0)
        decoded_targets = self.decoder(encoded_hidden)
        
        shift_logits = decoded_targets[..., :-1, :].contiguous()
        shift_labels = inputs[..., 1:].contiguous()     
        loss = self.loss_fct( shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1) ) 
        return loss, decoded_targets

    
dimension = model.config.n_embd
bit_dim =  bittensor.__network_dim__
print(dimension,bit_dim)
print((bit_dim-dimension+1)//2, )
gp_server = server(model,dimension,bit_dim)

768 1024
128


In [10]:
optimizer = torch.optim.SGD(
    [ {"params": gp_server.parameters()} ],
    lr = 0.1,
    momentum = 0.9,
)
scheduler= torch.optim.lr_scheduler.StepLR(optimizer,
    step_size= 1.0,
    gamma=0.95
)

In [11]:
from qqdm import qqdm, format_str
from torch.nn.utils import clip_grad_norm_
for k in range(5):
    print("epoch:",k)
    epoch_loss = 0
    epoch_batches = dataload.dataloader(epoch_length=100)
    for iteration, inputs in enumerate(epoch_batches):
        optimizer.zero_grad()
        loss, _ = gp_server( inputs )
        loss.backward()
        clip_grad_norm_(gp_server.parameters(), 1.0)
        optimizer.step()
        epoch_loss += loss.item()
        if iteration % 10 == 1:
            print(loss.item())
    print(loss)
    print(epoch_loss/100)
    print(optimizer.param_groups[0]['lr'])

epoch: 0
12.664243698120117
8.308642387390137
6.936455249786377
6.516313076019287
6.17348575592041
5.950094223022461
5.6457719802856445
4.744077682495117
4.666168212890625
3.5137765407562256
tensor(2.9335, grad_fn=<NllLossBackward>)
6.077446804046631
0.1
epoch: 1
7.911247253417969
6.554608345031738
5.300604343414307
4.340009689331055
3.1945810317993164
2.3247289657592773
1.7959849834442139
1.3939138650894165
2.0052690505981445
1.335225224494934
tensor(1.0806, grad_fn=<NllLossBackward>)
2.9557995969057083
0.1
epoch: 2
9.672611236572266
6.961157321929932
5.796114444732666
4.919164180755615
2.8587088584899902
2.5214738845825195
1.7089859247207642
1.0534708499908447
1.2759208679199219
0.7316533327102661
tensor(0.5866, grad_fn=<NllLossBackward>)
3.24380463719368
0.1
epoch: 3
2021-09-17 12:33:07.112 |     SUCCESS      | Retrieving a dataset files from the IPFS gateway...
2021-09-17 12:33:07.556 |     SUCCESS      | Loaded folder:      QmXwfPoh2QFYqC6cYcW8kzyd9ruFfhnUi2kVBkdhawjUzj
2021-09-17

In [22]:
metagraph = bittensor.metagraph().sync()

100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:07<00:00,  1.11s/it]


In [23]:
den = bittensor.dendrite()

In [24]:
representations, _ = bittensor.dendrite().forward_text (
    endpoints = metagraph.endpoints,
    inputs = "The quick brown fox jumped over the lazy dog"
)

2021-09-13 10:53:37.084 |     Connect      | uid: 0   |   /ipv4/99.238.136.56:8091  | hotkey:5CvfEfVYVQxBzpDWbb7HT5vtFWkAfk6x5h9V7doBGsthhrto | coldkey:5Evp4NftWF4eXsE38JEj3QM1aKEC3LxkuMTjZmfeSvFuhrkU 
2021-09-13 10:53:37.219 |     Connect      | uid: 1   |   /ipv4/99.238.136.56:8091  | hotkey:5CSgBX5uWQSniycimEJzc88LJuD3sUyHVWwkAicm6jh7EPii | coldkey:5Evp4NftWF4eXsE38JEj3QM1aKEC3LxkuMTjZmfeSvFuhrkU 
2021-09-13 10:53:37.348 |     Connect      | uid: 2   |  /ipv4/190.233.207.95:8091  | hotkey:5EkF63UTYSFsk8gRRi4cHPN56CV6XcD2GsdnLSzsYmFH4kKq | coldkey:5GRfTDpPckkUWCsBFq5rwvhTHDTWZfcyBJC5x69KYzSsA7ws 
2021-09-13 10:53:37.470 |     Connect      | uid: 3   |   /ipv4/70.69.22.235:8091   | hotkey:5F3wYX1ErB1ZnjH8rNyMBCfQ3MnRvvc25ffqgFQAEFrLBTZh | coldkey:5Gpcc7EJDxXTXgmyJUU84x1Nb44GZN69McYTx7UKjJUYo8eS 
2021-09-13 10:53:37.596 |     Connect      | uid: 4   |   /ipv4/70.69.22.235:8091   | hotkey:5EXVN93g6Y1PqppgKMVE3sEMURWkf2D7yJ2xsbgu1o7mCm6L | coldkey:5Gpcc7EJDxXTXgmyJUU84x1Nb44GZN69McYTx7UK

2021-09-13 10:53:41.775 |     Connect      | uid: 38  |  /ipv4/73.138.252.113:8091  | hotkey:5FkJ8tTwQ8bLd7iUzhZHL5dUELLCdHHhM6Vocdx2bomExv1B | coldkey:5HGfzDVCs73qicaUirZ5Xd2FVsXen83TYAh5bA3WY4GfSZzu 
2021-09-13 10:53:41.901 |     Connect      | uid: 39  |  /ipv4/73.138.252.113:8091  | hotkey:5EM5XiR2aTxBdyyak3f3xgN6MRFnNLKic69s4vuZh3wjYoTP | coldkey:5ELsrhbBw4hFNyVBLWxpsAC1B5d99sX52nXzPiCTW1jdLYK9 
2021-09-13 10:53:42.028 |     Connect      | uid: 40  |   /ipv4/200.60.96.129:8091  | hotkey:5GNA2Wdn93zMKotZo3i9DQ2hqFd33qV59SmGX1if5cEMzoLj | coldkey:5FZbVpr4xdaQoQynBWDzKjpAutf1MHXRqpqZ2VdqPe3sUJV5 
2021-09-13 10:53:42.166 |     Connect      | uid: 41  |  /ipv4/143.198.168.228:8091 | hotkey:5HC3yDi2hP4y7zSvKKuggP7AQbrF5HKQNNfadcA3EAzZ6UFD | coldkey:5Hb1Zpgy7Qkj7czEekMU7R7KN2sMe43rNwZmUCbJxQpA2BT7 
2021-09-13 10:53:42.292 |     Connect      | uid: 42  |  /ipv4/67.205.158.228:8091  | hotkey:5FX4SEsn1hDMR2xyQZ9fbCu8y2LsdSQ1yXpq62b6dBKMtRT6 | coldkey:5H8Yt5nR64T52uFeDgF3n5aJT4r8NhPJsgbokHWE

2021-09-13 10:53:44.129 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5CUSv6rbLzJfFRyHWZQppwJGLDFVoEVofpR7mTiynYAeWzVk | None 
2021-09-13 10:53:44.130 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5HRH9zP4pdyHybRmzPrDQk3XAmega9K2gaPCgYqoUeVNKrDV | None 
2021-09-13 10:53:44.132 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5DrGFsXzj1mBRU6y9h1BpdjQ2Li1yVzYrCkrvfT371R2jZ4j | None 
2021-09-13 10:53:44.133 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5DaXNZJ73Yh8HhAGXcnJRCgPyD76X5mnQfAcjzseVgnazYpD | None 
2021-09-13 10:53:44.136 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5HNQ2HkJxgtQyq7xP4FKkm9uY1AyXE7WqzowJWgdCftRT2MU | None 
2021-09-13 10:53:44.136 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5Gn4WagFnJ9Rt71vRtW3R2WAgBmHpVtivNCKt4i3x67znwNw | None 
2021-09-13 10:53:44.137 |     Success      | Dendrite | Forward  | ---> |      [1, 9]     | 5EU8SUzjWugikE

2021-09-13 10:53:49.094 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5CvfEfVYVQxBzpDWbb7HT5vtFWkAfk6x5h9V7doBGsthhrto | grpc.StatusCode.DEADLINE_EXCEEDED 
2021-09-13 10:53:49.097 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5F3wYX1ErB1ZnjH8rNyMBCfQ3MnRvvc25ffqgFQAEFrLBTZh | grpc.StatusCode.DEADLINE_EXCEEDED 
2021-09-13 10:53:49.108 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5FC6iTwsR3oJs4db92uBTjPBs6HuA1H4bPCs6aGbvdLxa88m | grpc.StatusCode.DEADLINE_EXCEEDED 
2021-09-13 10:53:49.117 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5Dcs8rsAiYuo6sH95AaJ5VcvtAkUyvWzRov3My5XrA3hfvbX | grpc.StatusCode.DEADLINE_EXCEEDED 
2021-09-13 10:53:49.118 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5CFoVMAiiNEM8cSujD8pDjZLnjkjAtVhrqKSgpRb1uM9qEhA | grpc.StatusCode.DEADLINE_EXCEEDED 
2021-09-13 10:53:49.121 |     Timeout      | Dendrite | Forward  | <--- |       [x]       | 5H3aaanYf148Bw9Rcw

In [25]:
for i ,j in enumerate(representations):
    print(i)
    print(torch.sum(j))

0
tensor(0., grad_fn=<SumBackward0>)
1
tensor(0., grad_fn=<SumBackward0>)
2
tensor(0., grad_fn=<SumBackward0>)
3
tensor(0., grad_fn=<SumBackward0>)
4
tensor(0., grad_fn=<SumBackward0>)
5
tensor(0., grad_fn=<SumBackward0>)
6
tensor(9.8909, grad_fn=<SumBackward0>)
7
tensor(-8.9785, grad_fn=<SumBackward0>)
8
tensor(-1.0479, grad_fn=<SumBackward0>)
9
tensor(6.3517, grad_fn=<SumBackward0>)
10
tensor(-1.5079, grad_fn=<SumBackward0>)
11
tensor(0., grad_fn=<SumBackward0>)
12
tensor(16.0985, grad_fn=<SumBackward0>)
13
tensor(0., grad_fn=<SumBackward0>)
14
tensor(0., grad_fn=<SumBackward0>)
15
tensor(0., grad_fn=<SumBackward0>)
16
tensor(0., grad_fn=<SumBackward0>)
17
tensor(0., grad_fn=<SumBackward0>)
18
tensor(0., grad_fn=<SumBackward0>)
19
tensor(0., grad_fn=<SumBackward0>)
20
tensor(0., grad_fn=<SumBackward0>)
21
tensor(0., grad_fn=<SumBackward0>)
22
tensor(0., grad_fn=<SumBackward0>)
23
tensor(0., grad_fn=<SumBackward0>)
24
tensor(0., grad_fn=<SumBackward0>)
25
tensor(0., grad_fn=<SumBackwa